# Sales Prediction _ Project 1 _ Part 5

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
pd.set_option('display.max_columns',None)
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn import set_config
set_config(transform_output="pandas")

In [ ]:
filepath="/content/drive/MyDrive/Coding Dojo/Data Sets/sales_predictions_2023.csv"
raw_df = pd.read_csv(filepath)
df = raw_df
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


# We will use the following Data Dictionary:

Variable  | Description
-------------------|------------------
Item_Identifier             | Unique product ID
Item_Weight                 | Weight of product
Item_Fat_Content            | Whether the product is low fat or regular
Item_Visibility             | The percentage of total display area of all products in a store allocated to the particular product
Item_Type                   | The category to which the product belongs
Item_MRP                    | Maximum Retail Price (list price) of the product
Outlet_Identifier           | Unique store ID
Outlet_Establishment_Year   | The year in which store was established
Outlet_Size                 | The size of the store in terms of ground area covered
Outlet_Location_Type        | The type of area in which the store is located
Outlet_Type                 | Whether the outlet is a grocery store or some sort of supermarket
Item_Outlet_Sales           | Sales of the product in the particular store. This is the target variable to be predicted.
---

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [ ]:
# Checking for duplicated rows
df.duplicated().sum()

0

In [ ]:
# Cheking for unique values
for col in df.columns:
  print(f"{col} : {df[col].unique()}")

Item_Identifier : ['FDA15' 'DRC01' 'FDN15' ... 'NCF55' 'NCW30' 'NCW05']
Item_Weight : [ 9.3    5.92  17.5   19.2    8.93  10.395 13.65     nan 16.2   11.8
 18.5   15.1   17.6   16.35   9.    13.35  18.85  14.6   13.85  13.
  7.645 11.65   5.925 19.25  18.6   18.7   17.85  10.     8.85   9.8
 13.6   21.35  12.15   6.42  19.6   15.85   7.39  10.195  9.895 10.895
  7.905  9.195  8.365  7.97  17.7   19.35   8.645 15.6   18.25   7.855
  7.825  8.39  12.85  19.     5.905  7.76  16.75  15.5    6.055  6.305
 20.85  20.75   8.895 19.7    8.75  13.3    8.31  19.75  17.1   10.5
  6.635 14.15   8.89   9.1    7.5   16.85   7.485 11.6   12.65  20.25
  8.6   12.6    8.88  20.5   13.5    7.235  6.92   8.02  12.8   16.6
 14.    16.    21.25   7.365 18.35   5.465  7.27   6.155 19.5   15.2
 14.5   13.1   12.3   11.1   11.3    5.75  11.35   6.525 10.3    5.78
 11.85  18.75   5.26  16.1    9.5   13.8   14.65   6.67   6.11  17.2
  6.32   4.88   5.425 14.1    7.55  17.25  12.    10.1    7.785 13.15
  8.5    

In [ ]:
# Adressing inconsistencies within "Item_Fat_Content"
df["Item_Fat_Content"]=df["Item_Fat_Content"].replace({'low fat':'Low Fat','LF':'Low Fat','reg':'Regular'})
df["Item_Fat_Content"].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [ ]:
# Cheking for missing values
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [ ]:
# grouping by "Outlet_Type" in order to identify NaNs of "Outlet_Size" in each "Outlet_Type"
df.groupby("Outlet_Type")["Outlet_Size"].value_counts(dropna=False)

Outlet_Type        Outlet_Size
Grocery Store      NaN             555
                   Small           528
Supermarket Type1  Small          1860
                   NaN            1855
                   High            932
                   Medium          930
Supermarket Type2  Medium          928
Supermarket Type3  Medium          935
Name: Outlet_Size, dtype: int64

In [ ]:
# grouping by "Outlet_Type" in order to identify NaNs of "Outlet_Size" in each "Outlet_Type"
df.groupby("Item_Type")["Item_Weight"].mean()

Item_Type
Baking Goods             12.277108
Breads                   11.346936
Breakfast                12.768202
Canned                   12.305705
Dairy                    13.426069
Frozen Foods             12.867061
Fruits and Vegetables    13.224769
Hard Drinks              11.400328
Health and Hygiene       13.142314
Household                13.384736
Meat                     12.817344
Others                   13.853285
Seafood                  12.552843
Snack Foods              12.987880
Soft Drinks              11.847460
Starchy Foods            13.690731
Name: Item_Weight, dtype: float64

The remaining NaNs of columns "Outlet_Size" and "Item_Weight" will be left as they are to be handled later in the ML part.

*ps : we will replace the missing values of outlet size by the most frequent value in the particalr outlet_type, and the missing item weights by the mean of each corresponding item_type.*

In [ ]:
#for _type in df["Item_Type"].value_counts().keys():                         # getting all types one by one to create a filter for each one within the for loop
#  _type_filter = df["Item_Type"] == _type                                    # creating a new filter to get the sub item Item Type category
#  avg_weight = df.loc[_type_filter,["Item_Weight"]].mean()[0]               # calculating average weight of sub Item Type category
#  df.loc[_type_filter & df["Item_Weight"].isna(),["Item_Weight"]]=avg_weight # filling missing values with the mean of the average weight of subcategory


In [ ]:
#display(df["Item_Weight"].describe(),df["Item_Weight"].isna().sum())

In [ ]:
#for _type in df["Outlet_Type"].value_counts().keys():                         # getting all types one by one to create a filter for each one within the for loop
#  _type_filter = df["Outlet_Type"] == _type                                    # creating a new filter to get the sub item Item Type category
#  most_freq_value = df.loc[_type_filter,["Outlet_Size"]].value_counts().keys()[0][0]        # calculating average weight of sub Item Type category
#  df.loc[_type_filter & df["Outlet_Size"].isna(),["Outlet_Size"]]=most_freq_value # filling missing values with the mean of the average weight of subcategory


#df["Outlet_Size"].value_counts(dropna=False)

In [ ]:
df.describe().round(2)

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.00,8523.00,8523.00,8523.00,8523.00
mean,12.86,0.07,140.99,1997.83,2181.29
std,4.64,0.05,62.28,8.37,1706.50
min,4.56,0.00,31.29,1985.00,33.29
25%,8.77,0.03,93.83,1987.00,834.25
50%,12.60,0.05,143.01,1999.00,1794.33
75%,16.85,0.09,185.64,2004.00,3101.30
max,21.35,0.33,266.89,2009.00,13086.96


In [ ]:
# splitting data
y=df["Item_Outlet_Sales"]
X=df.drop(columns=["Item_Outlet_Sales"])

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
X_train.shape

(6392, 11)

In [ ]:
df.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

In [ ]:
# Initiating list of numeric columns
num_cols=X_train.select_dtypes("number").columns
num_cols

Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year'],
      dtype='object')

In [ ]:
# Initiating list of ordinal columns
ord_cols=["Outlet_Size"]
ord_cols

['Outlet_Size']

In [ ]:
# Initiating list of nominal columns
nom_cols = X_train.select_dtypes("object").drop(columns=ord_cols).columns
nom_cols

Index(['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [ ]:
# numerical pipeline
num_nan_imputer = SimpleImputer(strategy="constant", fill_value=np.nan)
num_std_scaler = StandardScaler()
num_pipe=make_pipeline(num_nan_imputer,num_std_scaler)

# categorical(nominal) pipeline
nom_cns_imputer = SimpleImputer(strategy="constant", fill_value="MISSING")
nom_ohe_encoder = OneHotEncoder(sparse_output=False,handle_unknown="ignore")
nom_pipe=make_pipeline(nom_cns_imputer,nom_ohe_encoder)

# ordinal pipeline
ord_nan_imputer = SimpleImputer(strategy="constant", fill_value="MISSING")
ordered_categories=[["MISSING","Small","Medium","High"]]
ord_encoder = OrdinalEncoder(categories=ordered_categories)
ord_std_scaler = num_std_scaler
ord_pipe=make_pipeline(ord_nan_imputer,ord_encoder,ord_std_scaler)

In [ ]:
# Instantiating tupels for column transformer
num_tuple=('numeric',num_pipe,num_cols)
nom_tuple=('nominal',nom_pipe,nom_cols)
ord_tuple=('ordinal',ord_pipe,ord_cols)





In [ ]:
# instantiating column transformer
col_transformer=ColumnTransformer([num_tuple,nom_tuple,ord_tuple],verbose_feature_names_out=False,remainder="passthrough")

In [ ]:
# Fitting transformer
#col_transformer.fit(X_train)
nom_pipe.fit(X_train[nom_cols])

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='MISSING', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [ ]:
X_train[ord_cols].isna()

,Outlet_Size
4776,False
7510,False
5828,False
5327,False
4810,True
...,...
5734,True
5191,True
5390,True
860,True


In [ ]:
ord_cols=["Outlet_Size"]
ord_nan_imputer = SimpleImputer(strategy="constant", fill_value="MISSING")
ord_nan_imputer.fit(X_train[ord_cols])
ord_nan_imputer.transform(X_train[ord_cols])
X_train[ord_cols].isna().sum()

Outlet_Size    1812
dtype: int64

In [ ]:
ord_encoder.fit(X_train[ord_cols])
X_train[ord_cols]

ValueError: ignored

In [ ]:
ord_std_scaler.fit(X_train[ord_cols])
X_train[ord_cols]

ValueError: ignored